# Electricity Production Analysis Across 48 Countries (2010–2023): 
## Trends, Renewable Transition, and Energy Source Dynamics

## 1.0 About Dataset
### Dataset Summary:

This dataset contains raw electricity production data for 48 countries covering the period from 2010 to 2023. The data has not been pre-analyzed or interpreted and is intended to serve as a structured foundation for independent analysis and research.

It includes key variables such as country name, date, balance, product type, production value, and unit of measurement. These fields enable users to conduct their own analyses of electricity production over time, perform cross-country comparisons, and examine variations across different electricity products and balances.

As a raw dataset, it is suitable for time-series analysis, exploratory data analysis, and comparative studies related to electricity generation and energy systems. The dataset is particularly relevant for analysts, researchers, and policymakers seeking to derive insights on electricity production trends, energy system development, and sustainability transitions.

**Original Data Source:**
The primary dataset was sourced from IEA (2024) Monthly Electricity Statistics, International Energy Agency (IEA), Paris.

## 1.1	Data Preparation & Modeling (Python):
**Import, 
Clean and transform the raw dataset for analysis.**


In [10]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [12]:
df = pd.read_csv("C:/Users/DELL/Documents/Certified Data Analyst - XDi Course/Personal  learning data with Python/3. Global Electricity Prodcution Data/global_electricity_production_data.csv")

In [14]:
df.head()

,country_name,date,parameter,product,value,unit
0,Australia,12/1/2023,Net Electricity Production,Electricity,22646.1901,GWh
1,Australia,12/1/2023,Net Electricity Production,Total Combustible Fuels,13397.9356,GWh
2,Australia,12/1/2023,Net Electricity Production,"Coal, Peat and Manufactured Gases",9768.5223,GWh
3,Australia,12/1/2023,Net Electricity Production,Oil and Petroleum Products,289.5415,GWh
4,Australia,12/1/2023,Net Electricity Production,Natural Gas,3091.9272,GWh


In [16]:
# Finding out the overall data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121074 entries, 0 to 121073
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   country_name  121074 non-null  object 
 1   date          121074 non-null  object 
 2   parameter     121074 non-null  object 
 3   product       121074 non-null  object 
 4   value         121060 non-null  float64
 5   unit          121074 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.5+ MB


In [18]:
# Checking the summary statistics of all the columns (By default only summary stats of numerical columns are presented)
# df.describe()

# For summary stats of all columns, including none-numerical columns: 
df.describe(include="all")

,country_name,date,parameter,product,value,unit
count,121074,121074,121074,121074,121060.00000,121074
unique,48,168,7,16,NaN,1
top,France,3/1/2021,Net Electricity Production,Electricity,NaN,GWh
freq,3348,893,93318,34930,NaN,121074
mean,NaN,NaN,NaN,NaN,6925.08149,NaN
std,NaN,NaN,NaN,NaN,34224.45546,NaN
min,NaN,NaN,NaN,NaN,0.00000,NaN
25%,NaN,NaN,NaN,NaN,41.19900,NaN
50%,NaN,NaN,NaN,NaN,470.41900,NaN
75%,NaN,NaN,NaN,NaN,2629.71275,NaN


#### Checking for missing values

In [20]:
# Checking for missing values
df.isnull().sum()

country_name     0
date             0
parameter        0
product          0
value           14
unit             0
dtype: int64

In [22]:
# Check rows with missing values in 'value'
missing_rows = df[df['value'].isna()]
print(missing_rows[['country_name', 'date', 'parameter', 'product']])

     country_name       date parameter                           product
67          Chile  12/1/2023   Remarks  Data is estimated for this month
94     Costa Rica  12/1/2023   Remarks  Data is estimated for this month
285         Japan  12/1/2023   Remarks  Data is estimated for this month
804    Costa Rica  11/1/2023   Remarks  Data is estimated for this month
1517   Costa Rica  10/1/2023   Remarks  Data is estimated for this month
2230   Costa Rica   9/1/2023   Remarks  Data is estimated for this month
2944   Costa Rica   8/1/2023   Remarks  Data is estimated for this month
3659   Costa Rica   7/1/2023   Remarks  Data is estimated for this month
4375   Costa Rica   6/1/2023   Remarks  Data is estimated for this month
5090   Costa Rica   5/1/2023   Remarks  Data is estimated for this month
5805   Costa Rica   4/1/2023   Remarks  Data is estimated for this month
6523   Costa Rica   3/1/2023   Remarks  Data is estimated for this month
7242   Costa Rica   2/1/2023   Remarks  Data is est

#### Treament of Missing Values: Remove metadata rows (parameter = 'Remarks')

In [24]:
# Remove non-analytical metadata rows
df_clean = df[df['parameter'] != 'Remarks'].copy()

In [26]:
# Confirm no Remarks remain
assert 'Remarks' not in df_clean['parameter'].unique()

# Check remaining missing values
print(df_clean['value'].isna().sum())

0


In [28]:
df_clean.isnull().sum()

country_name    0
date            0
parameter       0
product         0
value           0
unit            0
dtype: int64

#### Treatment of Missing Values

Rows where <font color=blue> parameter = 'Remarks'</font> were excluded from the analytical dataset because they represent metadata annotations rather than measured electricity values. These entries indicate that reported figures for certain months are estimated and do not correspond to actual electricity production, consumption, import, or export metrics. After removing these metadata rows, the dataset was further validated to ensure that no missing numerical values remained in the value column. This preprocessing step guarantees analytical consistency and preserves the validity of all aggregations and trend analyses.

## 2. Data Analysis (SQL)

Following initial exploratory analysis and data cleaning, the dataset was migrated to a MySQL database to facilitate advanced querying and structured analysis. SQL was used to aggregate, filter, and analyze electricity production data across countries and time periods, enabling deeper insights into energy source distribution, production trends, and renewable energy adoption. This step supports scalable analysis and ensures analytical accuracy when addressing complex energy-related research questions.

### Code for MySQL

In [2]:
!pip install psycopg2-binary sqlalchemy

In [30]:
from sqlalchemy import create_engine

# MySQL connection
username = "root"
password = "Howfar22"
host = "localhost"
port = "3306"
database = "electricity_production"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# Write DataFrame to MySQL
table_name = "electricity_data" # choose any table name
df_clean.to_sql(table_name, engine, if_exists="replace", index=False)

# Read back sample
pd.read_sql("SELECT * FROM electricity_data LIMIT 5;", engine)

,country_name,date,parameter,product,value,unit
0,Australia,12/1/2023,Net Electricity Production,Electricity,22646.1901,GWh
1,Australia,12/1/2023,Net Electricity Production,Total Combustible Fuels,13397.9356,GWh
2,Australia,12/1/2023,Net Electricity Production,"Coal, Peat and Manufactured Gases",9768.5223,GWh
3,Australia,12/1/2023,Net Electricity Production,Oil and Petroleum Products,289.5415,GWh
4,Australia,12/1/2023,Net Electricity Production,Natural Gas,3091.9272,GWh


### Code for MySQL Server

In [33]:
!pip install pyodbc sqlalchemy

**Write DataFrame to SQL Server:**
- If you Selected Window Authentication during the creation of MySQL user account, there is no need for MySQL Server login and password, your windows credentials will be used.
- In the other hand, if you chose SQL Server Authentication, you will need to enter your SQL username and password.
- As we did not chose SQL Server Authentication, Code or DataFrame to SQL Server will not be necessary.